In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
from collections import Counter

In [ ]:
from tqdm import tqdm # permet d'afficher des barres de chargement 

In [ ]:
with open("bdd.txt","r") as f:
    num_folder = len(f.readlines())
with open("sbt.txt","r") as f:
    if (len(f.readlines()) != num_folder):
        error("[ERROR] number of BDD function does not match BDD number")
print("Number of function/Foler = ", num_folder)  # This would give length of files.

FileNotFoundError: ignored

In [ ]:
print("Get the bdd and sbt list via the bdd.txt & sbt.txt")
with open("bdd.txt") as f:
    tmp_bdd = f.read()
bdd = tmp_bdd.split("\n")

with open("sbt.txt") as f:
    tmp_sbt = f.read()
sbt = tmp_sbt.split("\n")

In [ ]:
# Cette fonction sera utilisée pour récupérer les codes sbt et bdd dans un autre code python
def return_bdd_sbt_and_num_folder():
    
    return bdd,sbt, num_folder

# Recuperation des valeurs max de BDD et SBT

Afin de creer une matrice d'encodage, il faut dans un premier temps determiner quelles seront les dimensions de cette matrice. Comme nous encodons en integer encoding les dimmensions des matrices vont etre les suivantes : ( nombre de fichiers, nombre de mot max par fichier ) 

La cellule ci dessous retourne le fichier ayant la plus grande longueur, donc qui possede le plus de mot :

In [ ]:
#Cette fonction calcule la taille de chaque code et renvoit le taille maximale. 
def return_max_len( text ):
    all_len = []
    for unique_code in text:
        code_split = unique_code.split(' ')
        all_len.append(len(code_split))
    return max(all_len)

# Separation variables / template 

Les counters permettent de compter le nombre d'occurence de chaque mot, ces données sont ensuite utilisé pour determiner si un mot appartient a la template ou si c est une variable.

In [ ]:
bdd_counter =Counter([word for sentence in bdd for word in sentence.split(' ')])
sbt_counter =Counter([word for sentence in sbt for word in sentence.split(' ')])

Une fois le nombre d'occurence de chaque mot calculé, il faut ensuite regarder quels mots sont les plus souvent répété. J'ai considéré que sur 5000 fichiers, si un mot apparait plus de 3500 fois, alors nous pouvons le considerer comme appartenant a la template. 

In [ ]:
'''
#Cette fonction retourne une liste contenant les mots les plus rencontrés
def return_common_words(counter):
    common_words = []
    for word in counter:
        occurence = counter[word]
        if occurence > 3500:
            common_words.append(word)
    return common_words
    
    
common_words_bdd = return_common_words(bdd_counter)
common_words_sbt = return_common_words(sbt_counter)
'''

Nous allons utiliser des token speciaux pour remplacer les variables, pour determiner le debut et la fin des sequences. 

In [ ]:
common_words_sbt=['', '(', 'MethodDeclaration', 'Modifier', 'public', ')',
                  'static', 'SimpleName', 'Parameter', 'PrimitiveType', 'int', 'boolean', 'BlockStmt',
                  'ExpressionStmt', 'VariableDeclarationExpr', 'VariableDeclarator', 'IntegerLiteralExpr',
                  'res', 'BinaryExpr', 'NameExpr', 'IfStmt', 'ReturnStmt', 'BooleanLiteralExpr', 'true',
                  'false', 'UnaryExpr']
common_words_bdd =  ['Feature:', 'Check', 'the', '', '#', 'result', 'to', 'True', 'Scenario:', 'Given', '=',
                     'and', 'When', 'i', 'call', 'Then', 'should', 'be', 'False']

common_words_bdd.append('<oov>')
common_words_sbt.append('<oov>')
common_words_bdd.append('sostoken ')
common_words_bdd.append(' eostoken')
common_words_bdd.append(' ')
common_words_sbt.append(' ')
'''
print('Common words SBT : \n',common_words_sbt)
print('\n')
print('Common words BDD : \n',common_words_bdd)
'''

Nous allons maintenant créer des dictionnnaires, c'est à dire associer à chaque mot un nombre. La cellule suivante creer deux dictionnaire pour chaque type de code ( BDD,SBT) qui nous premettent avec un mot d'acceder à son index et inversement.

In [ ]:
word_to_ind_sbt = {word : ind for ind,word in enumerate(common_words_sbt)}
ind_to_word_sbt = {ind : word for ind,word in enumerate(common_words_sbt)}

word_to_ind_bdd = {word : ind for ind,word in enumerate(common_words_bdd)}
ind_to_word_bdd = {ind : word for ind,word in enumerate(common_words_bdd)}

Afin d'encoder le BDD et SBT il faut creer des matrices, la cellule ci dessous permet de repertorier des données importantes pour leur création.

In [ ]:
number_folder = len(bdd)
max_words_sbt = return_max_len(sbt)+1
max_words_bdd = return_max_len(bdd)+1
number_of_words_sbt = len(common_words_sbt)
number_of_words_bdd = len(common_words_bdd)
'''
print(' number of forlder :',number_folder)
print(' number maximum of word in one sbt code :',max_words_sbt)
print(' number maximum of word in one bdd code :',max_words_bdd)
print(' number of different word in sbt :',number_of_words_sbt)
print(' number of different word in bdd :',number_of_words_bdd)
'''

# ---------------------------------------------------------------------------------------------------------------
# Génération des textes à trous

La fonction suivante permet de convertir un code SBT  ou BDD en un texte a trous, ou toutes les variables sont remplacées par des tokens ' < oov > ' ( out of vocabulary ) :

In [ ]:
def return_txt_trous(text,common_words):
    
    txt_trous = []
    
    
    for i,words in enumerate(text):
        temp_sentence = []
        split_words = words.split(' ')
        for j,word in enumerate(split_words):
            if word not in common_words:
                temp_sentence.append('<oov>')
                
            if word in common_words:
                temp_sentence.append(word)
                            
        txt_trous.append(' '.join(temp_sentence))
    return txt_trous

La focntion suivante permet de renvoyer un texte à trous mais pour un unique code SBT :

In [ ]:
def return_txt_trous_unique(text):
    
    txt_trous = []  
    split_words = text.split(' ')
    for j,word in enumerate(split_words):
        if word not in common_words_sbt:
            txt_trous.append('<oov>')

        if word in common_words_sbt:
            txt_trous.append(word)

    return ' '.join(txt_trous)

On génere les textes à trous avec les fonction précedentes

In [ ]:
txt_trou_unique = return_txt_trous_unique(sbt[0])

In [ ]:
txt_trou_bdd = return_txt_trous(bdd, common_words_bdd)
txt_trou_sbt = return_txt_trous(sbt, common_words_sbt)

On convertit les textes à trous en dataframe, nous allons ensuite leur appliquer des transformations.
Nous créons en plus deux nouvelles colonne dans notre dataframe, bdd_input et bdd_label. 

Nous allons donc donner au modele pour son entrainement en entrée le SBT et BDD_input, et en sortie le BDD_label que nous allons dans un premier temps convertir en numpy array

![image.png](attachment:image.png)

le fonctionnement d'un tel model est le suivant : 
Dans un premier temps les informations contenues dans le SBT sont condensée et transformée en un vecteur par l'encoder, ce vecteur est appelé le context vector. 

Le réseau de neuronne récurent qui prend le BDD_input en entrée et le BDD_Label en sortie, nous permet d'indiquer au modele qu'a la suite d'un certain mot, par exemple "je", il doit prédire "suis".

La couche d'attention permet au modele de se focaliser sur certaines partie de la sequence en entrée afin de prédire par exemple le début de la séquence en sortie.

En superposant ces information, le modele est donc capable de savoir que pour un contexte " i am a student" et a la suite de "< s >" il doit prédire " Je " 

< s > représente notre start of sentence token : sostoken

In [ ]:
df = pd.DataFrame(txt_trou_bdd, columns=['bdd'])
df['sbt'] = txt_trou_sbt
# pour l'instant bdd label et bdd inout sont identiques, nous allons ensuite rajouter les <sostoken> et les <eostoken>
df['bdd_input'] = txt_trou_bdd
df['bdd_label'] = txt_trou_bdd
df.tail(5) #permet d'afficher les 5 derniers éléments

Il faut maintenant convertir nos données en numpy array

In [ ]:
encoder_input = np.array(df.sbt)
decoder_input = np.array(df.bdd_input)
decoder_label = np.array(df.bdd_label)

On repartit nos données en set de test et set d'entrainnement, on choisit de consacrer 90% des données pour l'entrainement et 10% pour le test :

In [ ]:
total = num_folder
test_size = 0.1 #pourcentage des données de test 

train_encoder_input = encoder_input[:-int(total*test_size)]
train_decoder_input = decoder_input[:-int(total*test_size)]
train_decoder_label = decoder_label[:-int(total*test_size)]

test_encoder_input = encoder_input[-int(total*test_size):]
test_decoder_input = decoder_input[-int(total*test_size):]
test_decoder_label = decoder_label[-int(total*test_size):]
'''
print("train dataset shape")
print(train_encoder_input.shape)
print(train_decoder_input.shape)
print(train_decoder_label.shape)

print("\n\ntest dataset shape")
print(test_encoder_input.shape)
print(test_decoder_input.shape)
print(test_decoder_label.shape)
'''

# Encodage des données

Il faut maintenant encoder nos données, c'est a dire convertir ce texte à trous en une sequence de nombres, cette fonction prend en entrée notre texte à trous et des parametres pour l'encodage et renvoit une version encodée. 

In [ ]:
def encode(text,word_to_ind, dim1, dim2,type):# type 0 encoder, 1 decoder_input, 2 decoder_label
    encoded_matrix = np.full((dim1,dim2),word_to_ind[' '])
    #print('Encoding Matrix shape : ',encoded_matrix.shape)
    for i,words in enumerate(text):
        j=0
        #print(vars)
        split_word = words.split(' ')
        if type == 1:
            encoded_matrix[i,j]=word_to_ind['sostoken '] #on ajoute le start of sentence token
            j=j+1
        
        for word in (split_word):
            encoded_matrix[i,j]=word_to_ind[word]

            j=j+1
        if type == 2:
            encoded_matrix[i,j]=word_to_ind[' eostoken'] #on ajoute le end of sentence token
            j=j+1

    return encoded_matrix

La fonction suivante permet d'encoder un seul texte à trous. 

In [ ]:
def encode_unique(text):# type 0 encoder, 1 decoder_input, 2 decoder_label
    encoded_matrix_unique = np.full(256,word_to_ind_sbt[' '])

    split_word = text.split(' ')

    for i,word in enumerate(split_word):
        encoded_matrix_unique[i]=word_to_ind_sbt[word]

    return encoded_matrix_unique

In [ ]:
encode_unique(txt_trou_unique)

Génère les textes à trous encodés avec les fonction précédentes

In [ ]:
#print("train dataset shape")
train_encoder_input_encoded = encode(train_encoder_input, word_to_ind_sbt,len(train_encoder_input),max_words_sbt,0)
train_decoder_input_encoded = encode(train_decoder_input, word_to_ind_bdd,len(train_decoder_input),max_words_bdd,1)
train_decoder_label_encoded = encode(train_decoder_label, word_to_ind_bdd,len(train_decoder_label),max_words_bdd,2)
#print("\n\ntest dataset shape")

test_encoder_input_encoded = encode(test_encoder_input, word_to_ind_sbt,len(test_encoder_input),max_words_sbt,0)
test_decoder_input_encoded = encode(test_decoder_input, word_to_ind_bdd,len(test_decoder_input),max_words_bdd,1)
test_decoder_label_encoded = encode(test_decoder_label, word_to_ind_bdd,len(test_decoder_label),max_words_bdd,2)

In [ ]:
train_encoder_input_encoded.shape

la fonction suivante permet de décoder, donc de passer d'un texte encodé a un texte lisible

In [ ]:
def decode ( encoded_sequence, ind_to_char):
    decoded_sequence = []
    for var in encoded_sequence:
        decoded_sequence.append(ind_to_char[var])
    return ' '.join(decoded_sequence)

In [ ]:
decode(train_decoder_label_encoded[0],ind_to_word_bdd)

# Création du modèle

In [ ]:
from keras.layers import Input,Embedding,LSTM,Dense,Concatenate,Attention
from keras.models import Model
#from keras.utils import plot_model
from keras import backend as K

#hyperparameters
embedding_size = 256
hidden_size = 256

# trainer model (generator model will use the same encoder tho)
encoder_input = Input(shape=[max_words_sbt]) # size chain SBT 
encoder_embedding = Embedding(max_words_sbt,embedding_size,mask_zero=True)
encoder_embedded = encoder_embedding(encoder_input)

encoder_lstm1 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0)
encoder_output1,encoder_h1,encoder_c1 = encoder_lstm1(encoder_embedded)

encoder_lstm2 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0)
encoder_output2,encoder_h2,encoder_c2 = encoder_lstm2(encoder_output1)

encoder_lstm3 = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0)
encoder_output3,encoder_h3,encoder_c3 = encoder_lstm3(encoder_output1)

decoder_input = Input(shape=(None,))
decoder_embedding = Embedding(max_words_bdd,embedding_size,mask_zero=True) # len chain bdd
decoder_embedded = decoder_embedding(decoder_input)

decoder_lstm = LSTM(hidden_size,return_sequences=True,return_state=True,dropout=0.2,recurrent_dropout=0)
decoder_output,_,_ = decoder_lstm(decoder_embedded,initial_state=[encoder_h3,encoder_c3])

attn_layer = Attention()
attn_context = attn_layer([decoder_output,encoder_output3])

decoder_output = Concatenate(axis=-1)([decoder_output,attn_context])
tanh_dense= Dense(hidden_size*2,activation=K.tanh)
decoder_output = tanh_dense(decoder_output)

softmax_dense = Dense(number_of_words_bdd,activation='softmax')
decoder_output = softmax_dense(decoder_output)

trainer_model = Model([encoder_input,decoder_input],decoder_output)
trainer_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

La cellule suivante permet d'afficher un résumé du modele avec le nombre de neurone sur chaque couche.

In [ ]:
trainer_model.summary()

Entrainement du modèle :

In [ ]:
print("\nTraining of the model ... \n")
trainer_hist =trainer_model.fit([train_encoder_input_encoded,train_decoder_input_encoded],train_decoder_label_encoded,epochs=10,batch_size=128,validation_split=0.2)
print("\nTraining completed ! \n")


Dans le cas ou on entraine le modèle, la cellule suivante affiche la précision du modele en focntion du nombre d'epochs, ce graphe est utile pour prévenir de l'overfitting:

In [ ]:
print("\nSaving the model ... \n")
trainer_model.save_weights('template_attention_weigths.h5') # permet de sauvegarder les poids du modele
print("\nModel saved !\n")
print("\nPLoting the metrics / epochs ... \n")
losses = pd.DataFrame(trainer_model.history.history)
losses.plot()


In [ ]:
#trainer_model.load_weights('template_attention_weigths.h5') # permet de charger les poids

In [ ]:
#generator model
gen_encoder = Model([encoder_input],[encoder_output3,encoder_h3,encoder_c3])

gen_decoder_values_input = Input(shape=(max_words_sbt,hidden_size))
gen_decoder_h_input = Input(shape=[hidden_size])
gen_decoder_c_input = Input(shape=[hidden_size])

gen_decoder_embedded = decoder_embedding(decoder_input)
gen_decoder_output,gen_decoder_h,gen_decoder_c = decoder_lstm(gen_decoder_embedded,initial_state=[gen_decoder_h_input,gen_decoder_c_input])

attn_context = attn_layer([gen_decoder_output,gen_decoder_values_input])
gen_decoder_output = Concatenate(axis=-1)([gen_decoder_output,attn_context])

gen_decoder_output = tanh_dense(gen_decoder_output)
gen_decoder_output = softmax_dense(gen_decoder_output)

gen_decoder = Model([decoder_input]+[gen_decoder_values_input,gen_decoder_h_input,gen_decoder_c_input],[gen_decoder_output,gen_decoder_h,gen_decoder_c])

# Test

In [ ]:
def generate_from_encoder_input(encoder_input):
    encoder_input = encoder_input.reshape(1,-1)
    values,h,c = gen_encoder.predict(encoder_input)
    
    single_tok = np.zeros((1,1))
    single_tok[0,0] = word_to_ind_bdd['sostoken ']
    decoder_input = single_tok
    
    generated = []
    count = 0
    while(True):
        decoder_output,new_h,new_c = gen_decoder.predict([decoder_input]+[values,h,c])
        count +=1
        
        sampled_index = np.argmax(decoder_output[0,-1,:])
        sampled_word = ind_to_word_bdd[sampled_index]
        
        if sampled_word != ' eostoken' and sampled_index != 0:
            generated.append(sampled_word)
        if count >= max_words_bdd or sampled_word == ' eostoken':
            break
        
        h,c = new_h,new_c
        decoder_input[0,0] = sampled_index
    
    generated = ' '.join(generated)
    return generated

In [ ]:
'''
for i in range(520,525):
    print("\n<<sample encoder input SBT >>")
    print(decode(train_encoder_input_encoded[i],ind_to_word_sbt))
    print("\n")
    print("<<generated BDD >>")
    print(generate_from_encoder_input(train_encoder_input_encoded[i]))
    print("\n")
    print("<<Expected BDD>>")
    print(decode(train_decoder_label_encoded[i],ind_to_word_bdd))
    print("========================================\n")
'''

In [ ]:
input_encoded = test_encoder_input_encoded[0]

In [ ]:
def return_generated_template(input_encoded):
    generated = generate_from_encoder_input(input_encoded)
    generated = generated.split(' ')
    return generated

In [ ]:
print("\nEvaluating the model ... \n")
score = trainer_model.evaluate([train_encoder_input_encoded,train_decoder_input_encoded], train_decoder_label_encoded, verbose=0)
print("%s: %.2f%%" % (trainer_model.metrics_names[1], score[1]*100))